In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00


In [ ]:
!pip install -q einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00


In [ ]:

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
#dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] == 8 else torch.float16

model_id = "falcon-7B-instruct-600steps"
#model_id = "falcon-40B-instruct-600steps"
config = PeftConfig.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        return_dict=True,
        device_map = "auto",
        #use_ram_optimized_load=False,
        offload_folder="./tmp",
        #torch__dtype = dtype,
        #offload_dir = '.',
        #load_in_8bit = True,
        #device_map = 'auto',
        trust_remote_code = True)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(model, model_id, offload_dir="./tmp")

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict(

In [ ]:
m = model.merge_and_unload()

In [ ]:
m

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear(in_features=4544, out_features=4672, bias=False)
          (dense): Linear(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)

In [ ]:
#m.push_to_hub('falcon-7B-instruct-300steps-merged', use_auth_token=True, create_pr=1)
m.push_to_hub('falcon-7B-instruct-600steps-merged', use_auth_token=True, create_pr=1)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:830: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.78G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Pooja96/falcon-7B-instruct-600steps-merged/commit/29dd89979d689739638d17caa8f1d8c0aebf81ff', commit_message='Upload RWForCausalLM', commit_description='', oid='29dd89979d689739638d17caa8f1d8c0aebf81ff', pr_url='https://huggingface.co/Pooja96/falcon-7B-instruct-600steps-merged/discussions/1', pr_revision='refs/pr/1', pr_num=1)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
#tokenizer.push_to_hub('/falcon-7B-instruct-300steps-merged', use_auth_token=True, create_pr=1)
tokenizer.push_to_hub('/falcon-7B-instruct-600steps-merged', use_auth_token=True, create_pr=1)


CommitInfo(commit_url='https://huggingface.co/Pooja96/falcon-7B-instruct-600steps-merged/commit/84312596284611b53c8c0b26f81d489c72fc20fb', commit_message='Upload tokenizer', commit_description='', oid='84312596284611b53c8c0b26f81d489c72fc20fb', pr_url='https://huggingface.co/Pooja96/falcon-7B-instruct-600steps-merged/discussions/2', pr_revision='refs/pr/2', pr_num=2)